In [3]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
import pandas as pd
import numpy as np
import acquire1
import re
import unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

from env import github_token, github_username


From reading the script documentation it seems that the acquire is taken care of and pulls in a JSON data file that hold all of the data.

From here I will need to import the results and see what I can do with them

In reviewing the material inside of the JSON file I am going to need to declutter the script.
- There are many free floating characters present 
- Turn this clean data into a pandas dataframe

<a href="https://ds.codeup.com/nlp/explore/">This chapter</a>

In [4]:
df = pd.read_json("data.json")
df

,repo,language,readme_contents
0,dickreuter/Poker,Python,"Deep mind pokerbot for pokerstars, partypoker ..."
1,pokerregion/poker,Python,Poker framework for Python\n==================...
2,ishikota/PyPokerEngine,Python,# PyPokerEngine\n\n[![Build Status](https://tr...
3,andrewprock/pokerstove,C++,PokerStove\n==========\n\n[![OpenHUB Metrics](...
4,mdp/JsPoker,JavaScript,![JS Poker](http://img.mdp.im.s3.amazonaws.com...
5,Jeremiah9000/Poker-with-Python,Python,# Python Poker\nTexas Holdem Poker made with P...
6,ginuerzh/poker,JavaScript,webpoker\n========\n\na poker game use phaser ...
7,rundef/node-poker-odds-calculator,TypeScript,# Poker Odds Calculator\n\n[![Build Status](ht...
8,datamllab/rlcard,Python,# RLCard: A Toolkit for Reinforcement Learning...
9,Wizehive/firepoker,JavaScript,# Agile Planning Poker® powered by Firebase an...


Restarted approach to getting all of the text formatted

In [5]:
df.readme_contents

0     Deep mind pokerbot for pokerstars, partypoker ...
1     Poker framework for Python\n==================...
2     # PyPokerEngine\n\n[![Build Status](https://tr...
3     PokerStove\n==========\n\n[![OpenHUB Metrics](...
4     ![JS Poker](http://img.mdp.im.s3.amazonaws.com...
5     # Python Poker\nTexas Holdem Poker made with P...
6     webpoker\n========\n\na poker game use phaser ...
7     # Poker Odds Calculator\n\n[![Build Status](ht...
8     # RLCard: A Toolkit for Reinforcement Learning...
9     # Agile Planning Poker® powered by Firebase an...
10    Poker Hand History Parser\n===================...
Name: readme_contents, dtype: object

In [8]:
bd = df.readme_contents

In [6]:
type(df.readme_contents)

pandas.core.series.Series

In [9]:
bd = re.sub(r"[^a-z0-9'\s]", '', bd)

TypeError: expected string or bytes-like object

Error code above indicates that re.sub can only take in string or byte-like objects:

Can run through a for loop with the chnages being applied

In [11]:
bb = df.readme_contents[0]

In [13]:
bb = re.sub(r"[^a-z0-9'\s]", '', bb)
bb

"eep mind pokerbot for pokerstars partypoker and oker\n\n\nhis pokerbot plays automatically on okerstars artypoker and  oker ny other table can be mapped as well\nt works with image recognition montecarlo simulation and a basic genetic algorithm\nhe mouse is moved automatically and the bot can potentially play for hours based on a large number of parameters\n\nou can download the binaries and run the executable directly from here\nhttpssourceforgenetprojectspartypokerpokerstarspokerbot\n\nunning the bot\n\n\nartypoker\n\n se ast orward ables\n elect fficial artyoker table scraper\n\n figure docpartypokergif\n\nokerstars\n\n se oom ables\n elect fficial oker tars table scraper\n or okerstars you will need to make some adjustments to the client t needs to look exactly as the below in order to run it with the fficial okerstars able scraper\n ake sure to match the card styles 4 color covered card styles and table styles\n\n figure docpsexamplepng\n\noker\n\nake sure your oker setup looks a

In [14]:
# we will define a basic_clean function for a single document (one string)
def basic_clean(string):
    '''
    This function takes in a string and
    returns the string normalized.
    '''
    # we will normalize our data into standard NFKD unicode, feed it into an ascii encoding
    # decode it back into UTF-8
    string = unicodedata.normalize('NFKD', string)\
             .encode('ascii', 'ignore')\
             .decode('utf-8', 'ignore')
    # utilize our regex substitution to remove our undesirable characters, then lowercase
    string = re.sub(r"[^\w0-9'\s]", '', string).lower()
    return string

In [16]:
bb = basic_clean(bb)
bb
#

"eep mind pokerbot for pokerstars partypoker and oker\n\n\nhis pokerbot plays automatically on okerstars artypoker and  oker ny other table can be mapped as well\nt works with image recognition montecarlo simulation and a basic genetic algorithm\nhe mouse is moved automatically and the bot can potentially play for hours based on a large number of parameters\n\nou can download the binaries and run the executable directly from here\nhttpssourceforgenetprojectspartypokerpokerstarspokerbot\n\nunning the bot\n\n\nartypoker\n\n se ast orward ables\n elect fficial artyoker table scraper\n\n figure docpartypokergif\n\nokerstars\n\n se oom ables\n elect fficial oker tars table scraper\n or okerstars you will need to make some adjustments to the client t needs to look exactly as the below in order to run it with the fficial okerstars able scraper\n ake sure to match the card styles 4 color covered card styles and table styles\n\n figure docpsexamplepng\n\noker\n\nake sure your oker setup looks a

In [18]:
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to
[nltk_data]     /Users/lelouchlamprerouge/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [19]:
bb = unicodedata.normalize('NFKD', bb)\
    .encode('ascii', 'ignore')\
    .decode('utf-8', 'ignore')

print(bb[0:500])


eep mind pokerbot for pokerstars partypoker and oker


his pokerbot plays automatically on okerstars artypoker and  oker ny other table can be mapped as well
t works with image recognition montecarlo simulation and a basic genetic algorithm
he mouse is moved automatically and the bot can potentially play for hours based on a large number of parameters

ou can download the binaries and run the executable directly from here
httpssourceforgenetprojectspartypokerpokerstarspokerbot

unning the bot





In [21]:
bb = bb.replace("\n","")
bb

"eep mind pokerbot for pokerstars partypoker and okerhis pokerbot plays automatically on okerstars artypoker and  oker ny other table can be mapped as wellt works with image recognition montecarlo simulation and a basic genetic algorithmhe mouse is moved automatically and the bot can potentially play for hours based on a large number of parametersou can download the binaries and run the executable directly from herehttpssourceforgenetprojectspartypokerpokerstarspokerbotunning the botartypoker se ast orward ables elect fficial artyoker table scraper figure docpartypokergifokerstars se oom ables elect fficial oker tars table scraper or okerstars you will need to make some adjustments to the client t needs to look exactly as the below in order to run it with the fficial okerstars able scraper ake sure to match the card styles 4 color covered card styles and table styles figure docpsexamplepngokerake sure your oker setup looks as follows when using it with the fficial oker able figure docg

From here I need to drop off both words not in the dictionary, and incomplete words

In [22]:
sent = bb
" ".join(w for w in nltk.wordpunct_tokenize(sent) \
         if w.lower() in words or not w.isalpha())
# 'Io to the beach with my'

"mind for and automatically on and other table can be as works with image recognition simulation and a basic genetic mouse is automatically and the bot can potentially play for based on a large number of can the and run the executable directly from the se ast elect table scraper figure se elect tars table scraper or you will need to make some to the client t needs to look exactly as the below in order to run it with the able scraper ake sure to match the card 4 color covered card and table figure sure your setup as when it with the able figure docggpk2pngeneral you just want to run the bot please do the the of the bot from here can then immediately run the bot t will analyze the poker client move the mouse and play in one of its of the are based on zoom or ast are welcome to edit and any to the play of the bot for your most it may be useful to put the poker client inside a virtual machine to avoid it from the bot the bot will then be able to control the mouse inside the virtual machine

In [35]:
type(bb)

str

In [54]:
print(re.search(r"https\w*\s", bb))

<re.Match object; span=(418, 480), match='httpssourceforgenetprojectspartypokerpokerstarspo>


In [55]:
def show_all_matches(regexes, subject, re_length=6):
    print('Sentence:')
    print()
    print('    {}'.format(subject))
    print()
    print(' regexp{} | matches'.format(' ' * (re_length - 6)))
    print(' ------{} | -------'.format(' ' * (re_length - 6)))
    for regexp in regexes:
        fmt = ' {:<%d} | {!r}' % re_length
        matches = re.findall(regexp, subject)
        if len(matches) > 8:
            matches = matches[:8] + ['...']
        print(fmt.format(regexp, matches))


In [57]:
show_all_matches([r"https\w*\s"], bb)

Sentence:

    eep mind pokerbot for pokerstars partypoker and okerhis pokerbot plays automatically on okerstars artypoker and  oker ny other table can be mapped as wellt works with image recognition montecarlo simulation and a basic genetic algorithmhe mouse is moved automatically and the bot can potentially play for hours based on a large number of parametersou can download the binaries and run the executable directly from herehttpssourceforgenetprojectspartypokerpokerstarspokerbotunning the botartypoker se ast orward ables elect fficial artyoker table scraper figure docpartypokergifokerstars se oom ables elect fficial oker tars table scraper or okerstars you will need to make some adjustments to the client t needs to look exactly as the below in order to run it with the fficial okerstars able scraper ake sure to match the card styles 4 color covered card styles and table styles figure docpsexamplepngokerake sure your oker setup looks as follows when using it with the fficial oker ab

So from here I can find specifically the https website code,

that would be semantics some of these misspelled words could clue us in on where to go next

Snippets from the explore chapter both in the circulumn and excercise review

In [ ]:
ADDITIONAL_STOPWORDS = ['r', 'u', '2', 'ltgt']

def clean(text):
    'A simple function to cleanup text data'
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
    text = (unicodedata.normalize('NFKD', text)
             .encode('ascii', 'ignore')
             .decode('utf-8', 'ignore')
             .lower())
    words = re.sub(r'[^\w\s]', '', text).split()
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

In [ ]:
def clean(readme_contents):
    'A simple function to cleanup text data'
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english')
    readme_contents = (unicodedata.normalize('NFKD', readme_contents)
             .encode('ascii', 'ignore')
             .decode('utf-8', 'ignore')
             .lower())
    words = re.sub(r'[^\w\s]', '', text).split()
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

In [ ]:
df.head(3)

Ryan Mcall referenced his <a href="https://github.com/CodeupClassroom/jemison-nlp-exercises/blob/main/prepare_exercises.ipynb">exercise </a>

In [ ]:
df = clean(df)

In [ ]:
# we will define a basic_clean function for a single document (one string)
def basic_clean(string):
    '''
    This function takes in a string and
    returns the string normalized.
    '''
    # we will normalize our data into standard NFKD unicode, feed it into an ascii encoding
    # decode it back into UTF-8
    string = unicodedata.normalize('NFKD', string)\
             .encode('ascii', 'ignore')\
             .decode('utf-8', 'ignore')
    # utilize our regex substitution to remove our undesirable characters, then lowercase
    string = re.sub(r"[^\w0-9'\s]", '', string).lower()
    return string

In [ ]:
def tokenize(string):
    '''
    This function takes in a string and
    returns a tokenized string.
    '''
    # make our tokenizer, taken from nltk's ToktokTokenizer
    tokenizer = nltk.tokenize.ToktokTokenizer()
    # apply our tokenizer's tokenization to the string being input, ensure it returns a string
    string = tokenizer.tokenize(string, return_str = True)
    
    return string


In [ ]:


def stem(string):
    '''
    This function takes in a string and
    returns a string with words stemmed.
    '''
    # create our stemming object
    ps = nltk.porter.PorterStemmer()
    # use a list comprehension => stem each word for each word inside of the entire document,
    # split by the default, which are single spaces
    stems = [ps.stem(word) for word in string.split()]
    # glue it back together with spaces, as it was before
    string = ' '.join(stems)
    
    return string



In [ ]:
def lemmatize(string):
    '''
    This function takes in string for and
    returns a string with words lemmatized.
    '''
    # create our lemmatizer object
    wnl = nltk.stem.WordNetLemmatizer()
    # use a list comprehension to lemmatize each word
    # string.split() => output a list of every token inside of the document
    lemmas = [wnl.lemmatize(word) for word in string.split()]
    # glue the lemmas back together by the strings we split on
    string = ' '.join(lemmas)
    #return the altered document
    return string

In [ ]:
def remove_stopwords(string, extra_words = [], exclude_words = []):
    '''
    This function takes in a string, optional extra_words and exclude_words parameters
    with default empty lists and returns a string.
    '''
    # assign our stopwords from nltk into stopword_list
    stopword_list = stopwords.words('english')
    # utilizing set casting, i will remove any excluded stopwords
    stopword_list = set(stopword_list) - set(exclude_words)
    # add in any extra words to my stopwords set using a union
    stopword_list = stopword_list.union(set(extra_words))
    # split our document by spaces
    words = string.split()
    # every word in our document, as long as that word is not in our stopwords
    filtered_words = [word for word in words if word not in stopword_list]
    # glue it back together with spaces, as it was so it shall be
    string_without_stopwords = ' '.join(filtered_words)
    # return the document back
    return string_without_stopwords